<a href="https://colab.research.google.com/github/MiyoBran/Alura-ONE-G9/blob/main/formacion-Aprendiendo-a-hacer-ETL-G9-ONE/02-pandas-e-s-diferentes-formatos-archivo/ETL_02_Pandas_I_O_Excel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Chardet es una biblioteca de Python (y una herramienta de línea de comandos) utilizada para detectar automáticamente la codificación de caracteres de archivos de texto o secuencias de bytes, siendo ideal cuando se desconoce la codificación (ej. UTF-8, ASCII, ISO-8859). Soporta una amplia gama de codificaciones, incluyendo variantes de Unicode y asiáticas, devolviendo la codificación más probable con un nivel de confianza.

```
import chardet

# Leer el archivo en modo binario
with open('archivo.txt', 'rb') as f:
    datos = f.read()
    resultado = chardet.detect(datos)
    print(resultado)
# Salida: {'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}

```



##**Aula 2 - Leyendo archivos Excel**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##**Importando archivos Excel**

In [2]:
import pandas as pd

In [3]:
#datos = pd.read_csv('superstore_data.csv')
archivo = '/content/drive/MyDrive/Pandas/emisiones_CO2.xlsx'
datos_co2 = pd.read_excel(archivo)

In [4]:
datos_co2.head()

,País,ISO 3166-1 alpha-3,Año,Total,Carbón,Aceite,Gas,Cemento,Quema,Otros,Per Capita
0,Afganistán,AFG,1750,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afganistán,AFG,1751,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afganistán,AFG,1752,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afganistán,AFG,1753,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afganistán,AFG,1754,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#Con este metodo puedo saber los nombres de las hojas del archivo excel
pd.ExcelFile(archivo).sheet_names

['emisiones_C02', 'emisiones_percapita', 'fuentes']

### 📊 Ingesta de Datos Tridimensionales: Archivos Excel (`.xlsx`)

**Concepto de Negocio:**
A diferencia de los archivos CSV (que son texto plano bidimensional), los archivos de Excel son "Libros" que contienen múltiples "Hojas" (Sheets). Esto añade una tercera dimensión a los datos. Al ingestar un Excel en Pandas, debemos ser explícitos sobre qué hoja queremos leer; de lo contrario, Pandas cargará únicamente la primera hoja por defecto.

**Optimización de Memoria (Clean Code):**
* **Mala Práctica:** Usar `pd.read_excel()` y luego `pd.ExcelFile()` por separado. Esto fuerza al sistema a abrir y descomprimir el archivo pesado dos veces.
* **Buena Práctica:** Crear un objeto `pd.ExcelFile` primero. Esto carga el archivo en la memoria caché del servidor una sola vez. Luego usamos ese objeto tanto para inspeccionar los nombres de las hojas como para extraer los datos mediante su método `.parse()`.

In [6]:
import pandas as pd

# 1. Definición de Rutas
RUTA_EXCEL_EMISIONES = '/content/drive/MyDrive/Pandas/emisiones_CO2.xlsx'

# 2. Ingesta Inteligente (Cargamos el "Libro" completo en memoria caché)
# Esto es mucho más eficiente que usar read_excel directamente
libro_excel = pd.ExcelFile(RUTA_EXCEL_EMISIONES)

# 3. Auditoría de Estructura (Verificamos qué hojas existen)
lista_hojas = libro_excel.sheet_names
print(f"📑 El archivo Excel contiene {len(lista_hojas)} hojas:")
print(lista_hojas)
print("-" * 40)

# 4. Extracción de Datos
# Por ahora extraemos la primera hoja (índice 0 o por su nombre exacto)
# El método .parse() extrae la tabla desde el libro que ya está en memoria
df_emisiones = libro_excel.parse(lista_hojas[0])

# 5. Inspección visual
display(df_emisiones.head())

📑 El archivo Excel contiene 3 hojas:
['emisiones_C02', 'emisiones_percapita', 'fuentes']
----------------------------------------


,País,ISO 3166-1 alpha-3,Año,Total,Carbón,Aceite,Gas,Cemento,Quema,Otros,Per Capita
0,Afganistán,AFG,1750,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afganistán,AFG,1751,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afganistán,AFG,1752,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afganistán,AFG,1753,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afganistán,AFG,1754,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### **Parámetros de la función read_excel**

Documentación: https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html

### 📂 Diversidad de Formatos: Más allá del `.xlsx`

**Concepto de Negocio:**
En el mundo real corporativo, los datos no siempre llegan en el formato más moderno. Los sistemas heredados (Legacy Systems), las políticas de seguridad y las normativas de software libre obligan al Científico de Datos a saber procesar múltiples extensiones de hojas de cálculo.

**Diccionario de Extensiones:**
* **`.xls` (Legacy):** Formato antiguo (pre-2007). Basado en binario.
* **`.xlsx` (Estándar Actual):** Basado en XML. Es el estándar de la industria.
* **`.xlsm` (Macros):** Contiene scripts ejecutables (VBA). *Nota de Seguridad:* Pandas extrae los datos de forma segura ignorando las macros, lo cual es ideal para evitar inyecciones de código malicioso.
* **`.xlsb` (Binario Optimizado):** Guarda los datos de forma binaria en lugar de XML. Es significativamente más rápido para leer y escribir. Ideal para archivos masivos (Ej. +1 millón de filas).
* **`.ods` / `.odf` (Open Source):** Formato de documento abierto (LibreOffice, OpenOffice). Muy común en la administración pública.

**🛠️ Tip de Arquitectura de Software (Engines):**
Aunque la función siempre es `pd.read_excel()`, Pandas requiere motores específicos para procesar formatos que no sean `.xlsx`. Si te enfrentas a un archivo especial, recuerda usar el parámetro `engine`:
* Para `.xlsb` -> `engine='pyxlsb'`
* Para `.ods` -> `engine='odf'`
* Para `.xls` -> `engine='xlrd'`

In [7]:
percapita = pd.read_excel(archivo, sheet_name='emisiones_percapita')

In [8]:
#tambien podemos usar el metodo tail() en vez de head()
percapita.tail()

,País,ISO 3166-1 alpha-3,Año,Total,Carbón,Aceite,Gas,Cemento,Quema,Otros
63099,Global,WLD,2017,4749682.0,1908857.0,1610910.0,940144.0,198416.0,51579.0,39776.0
63100,Global,WLD,2018,4792753.0,1919213.0,1596350.0,979965.0,204225.0,53634.0,39366.0
63101,Global,WLD,2019,4775633.0,1896468.0,1589920.0,984878.0,208309.0,56569.0,39490.0
63102,Global,WLD,2020,4497423.0,1807760.0,1427353.0,963695.0,208844.0,51981.0,37789.0
63103,Global,WLD,2021,4693699.0,1893923.0,1496614.0,1001585.0,211472.0,52663.0,37443.0


In [9]:
#alternativamente podemos tomar registros aleatorios con sample(int)
percapita.sample(8)

,País,ISO 3166-1 alpha-3,Año,Total,Carbón,Aceite,Gas,Cemento,Quema,Otros
8858,Burundi,BDI,1904,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32005,Liechtenstein,LIE,1931,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29548,Incendios Petroleros En Kuwait,NaN,1922,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23835,Guayana,GUY,1921,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12339,Costa Rica,CRI,1849,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10305,Chad,TCD,1991,63670.0,0.0,63670.0,0.0,NaN,0.0,NaN
59698,Tanzania,TZA,1880,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13064,Cuba,CUB,1758,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
fuentes = pd.read_excel(archivo, sheet_name='fuentes')


In [11]:
fuentes.tail()

,País,ISO 3166-1 alpha-3,Año,Total,Carbón,Aceite,Gas,Cemento,Quema,Otros,Per Capita
63099,Global,WLD,2017,"CDIAC 2022, BP, and Sum of countries",CDIAC 2022 and BP,CDIAC 2022 and BP,CDIAC 2022 and BP,Andrew cement,CDIAC 2022 and GCP,[NONE],"CDIAC 2022, BP, Sum of countries, and UN popul..."
63100,Global,WLD,2018,"CDIAC 2022, BP, and Sum of countries",CDIAC 2022 and BP,CDIAC 2022 and BP,CDIAC 2022 and BP,Andrew cement,CDIAC 2022 and GCP,[NONE],"CDIAC 2022, BP, Sum of countries, and UN popul..."
63101,Global,WLD,2019,"CDIAC 2022, BP, and Sum of countries",CDIAC 2022 and BP,CDIAC 2022 and BP,CDIAC 2022 and BP,Andrew cement,CDIAC 2022 and GCP,[NONE],"CDIAC 2022, BP, Sum of countries, and UN popul..."
63102,Global,WLD,2020,"CDIAC 2022, BP, and Sum of countries",CDIAC 2022 and BP,CDIAC 2022 and BP,CDIAC 2022 and BP,Andrew cement,CDIAC 2022 and GCP,[NONE],"CDIAC 2022, BP, Sum of countries, and UN popul..."
63103,Global,WLD,2021,"CDIAC 2022, BP, and Sum of countries",CDIAC 2022 and BP,CDIAC 2022 and BP,CDIAC 2022 and BP,Andrew cement,CDIAC 2022 and GCP,[NONE],"CDIAC 2022, BP, Sum of countries, and UN popul..."


In [12]:
#Veremos que tambien podemos solo guardar una x cantidad de lineas usando nrows
datos_primeras_lineas = pd.read_csv('/content/drive/MyDrive/Pandas/superstore_data.csv',nrows=5)
datos_primeras_lineas

,Id,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Response,Complain
0,1826,1970,Graduation,Divorced,84835,0,0,6/16/2014,0,189,...,111,189,218,1,4,4,6,1,1,0
1,1,1961,Graduation,Single,57091,0,0,6/15/2014,0,464,...,7,0,37,1,7,3,7,5,1,0
2,10476,1958,Graduation,Married,67267,0,1,5/13/2014,0,134,...,15,2,30,1,3,2,5,2,0,0
3,1386,1967,Graduation,Together,32474,1,1,11/5/2014,0,10,...,0,0,0,1,1,0,2,7,0,0
4,5371,1989,Graduation,Single,21474,1,0,8/4/2014,0,6,...,11,0,34,2,3,1,2,7,1,0


In [13]:
# Aqui el metodo que nos permite "seleccionar" solo algunas columnas, usando ":"
# ponemos el intervalo de columnas que queremos.
intervalo = pd.read_excel(archivo, sheet_name='emisiones_C02', usecols= 'A:D')

In [14]:
intervalo

,País,ISO 3166-1 alpha-3,Año,Total
0,Afganistán,AFG,1750,0.000000e+00
1,Afganistán,AFG,1751,0.000000e+00
2,Afganistán,AFG,1752,0.000000e+00
3,Afganistán,AFG,1753,0.000000e+00
4,Afganistán,AFG,1754,0.000000e+00
...,...,...,...,...
63099,Global,WLD,2017,3.609674e+10
63100,Global,WLD,2018,3.682651e+10
63101,Global,WLD,2019,3.708256e+10
63102,Global,WLD,2020,3.526409e+10


In [15]:
# tambien podemos usar ambos parametros a la vez, de manera que tenemos una
# cantidad de columnas y una cantidad de filas.
intervalo_2 = pd.read_excel(archivo, sheet_name='emisiones_C02', usecols= 'A:D', nrows=10)

In [16]:
intervalo_2

,País,ISO 3166-1 alpha-3,Año,Total
0,Afganistán,AFG,1750,0
1,Afganistán,AFG,1751,0
2,Afganistán,AFG,1752,0
3,Afganistán,AFG,1753,0
4,Afganistán,AFG,1754,0
5,Afganistán,AFG,1755,0
6,Afganistán,AFG,1756,0
7,Afganistán,AFG,1757,0
8,Afganistán,AFG,1758,0
9,Afganistán,AFG,1759,0


### 🕵️‍♂️ Auditoría de Datos y Filtrado Específico de Excel

**Concepto de Negocio:**
Al importar libros de Excel complejos, es obligatorio realizar un muestreo de calidad (QA) para verificar la integridad de los datos. Además, los proveedores suelen enviar planillas con decenas de columnas de cálculos intermedios que no necesitamos en nuestro sistema central.

**Herramientas de Muestreo:**
* `.head(n)`: Extrae los primeros 'n' registros.
* `.tail(n)`: Extrae los últimos 'n' registros (ideal para ver los datos más recientes).
* `.sample(n)`: Extrae 'n' registros aleatorios (ideal para detectar errores ocultos en el grueso de los datos).

**Filtrado Nativo de Excel:**
El parámetro `usecols` en Excel acepta la notación de rangos clásicos (ej. `'A:D'`). Combinado con `nrows`, nos permite extraer un "cuadrante" exacto de la hoja de cálculo de manera muy eficiente.

In [18]:
# Nota: Asumimos que la variable 'libro_excel' (pd.ExcelFile)
# ya está cargada en memoria desde la celda anterior.

# ---------------------------------------------------------
# 1. Auditoría de Calidad (Muestreo Aleatorio y Final)
# ---------------------------------------------------------
df_per_capita = libro_excel.parse('emisiones_percapita')

print("--- Auditoría QA: Últimos 5 registros (Tail) ---")
display(df_per_capita.tail())

print("\n--- Auditoría QA: 5 registros Aleatorios (Sample) ---")
display(df_per_capita.sample(5))


# ---------------------------------------------------------
# 2. Ingesta Filtrada (Cuadrante Específico)
# ---------------------------------------------------------
# Extraemos solo las columnas A, B, C y D, y apenas las primeras 10 filas.
df_emisiones_resumen = libro_excel.parse(
    'emisiones_C02',
    usecols='A:D',
    nrows=10
)

print("\n--- Resumen Gerencial (Filtrado por Rango A:D) ---")
display(df_emisiones_resumen)

--- Auditoría QA: Últimos 5 registros (Tail) ---


,País,ISO 3166-1 alpha-3,Año,Total,Carbón,Aceite,Gas,Cemento,Quema,Otros
63099,Global,WLD,2017,4749682.0,1908857.0,1610910.0,940144.0,198416.0,51579.0,39776.0
63100,Global,WLD,2018,4792753.0,1919213.0,1596350.0,979965.0,204225.0,53634.0,39366.0
63101,Global,WLD,2019,4775633.0,1896468.0,1589920.0,984878.0,208309.0,56569.0,39490.0
63102,Global,WLD,2020,4497423.0,1807760.0,1427353.0,963695.0,208844.0,51981.0,37789.0
63103,Global,WLD,2021,4693699.0,1893923.0,1496614.0,1001585.0,211472.0,52663.0,37443.0



--- Auditoría QA: 5 registros Aleatorios (Sample) ---


,País,ISO 3166-1 alpha-3,Año,Total,Carbón,Aceite,Gas,Cemento,Quema,Otros
37107,Montserrat,MSR,1865,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36550,Mongolia,MNG,1852,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22084,Groenlandia,GRL,1802,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53184,San Cristóbal Y Nieves-Anguila,KNA,1894,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19885,Polinesio Francés,PYF,1779,NaN,NaN,NaN,NaN,NaN,NaN,NaN



--- Resumen Gerencial (Filtrado por Rango A:D) ---


,País,ISO 3166-1 alpha-3,Año,Total
0,Afganistán,AFG,1750,0
1,Afganistán,AFG,1751,0
2,Afganistán,AFG,1752,0
3,Afganistán,AFG,1753,0
4,Afganistán,AFG,1754,0
5,Afganistán,AFG,1755,0
6,Afganistán,AFG,1756,0
7,Afganistán,AFG,1757,0
8,Afganistán,AFG,1758,0
9,Afganistán,AFG,1759,0


### ⏱️ Auditoría de Rendimiento: I/O vs. CPU (El costo de leer Excel)

**Observación de Arquitectura:**
Aunque utilizamos `pd.ExcelFile` para cargar el libro en la memoria caché, la extracción de la hoja de cálculo (`.parse()`) demoró aproximadamente 5 segundos. ¿Por qué no fue instantáneo si ya estaba en la RAM?

En términos de sistemas, debemos separar dos operaciones que consumen tiempo:
1. **El cuello de botella I/O (Entrada/Salida):** Es el tiempo que tarda el servidor en ir a buscar el archivo físico al disco duro (o a Google Drive a través de la red) y traerlo a la memoria RAM. `pd.ExcelFile` ejecuta este paso **una sola vez**. (Es como traer la caja fuerte del archivo muerto a tu escritorio).
2. **El cuello de botella de la CPU (Procesamiento):** Los archivos `.xlsx` son en realidad archivos comprimidos (`.zip`) llenos de documentos de texto en lenguaje XML. Cuando ejecutamos `.parse()`, el motor de Python (`openpyxl`) tiene que leer línea por línea ese XML y transformarlo matemáticamente en una tabla estructurada (DataFrame) de Pandas.

[Image of pandas ExcelFile memory caching and XML parsing]

**Conclusión Gerencial:**
Ese trabajo de traducción XML para más de 63,000 registros es un proceso intensivo para el procesador (CPU), y de ahí provienen esos 5 segundos. Sin embargo, gracias a nuestra optimización, evitamos repetir el costoso "viaje de red" inicial que el motor habría hecho si usábamos la función básica múltiples veces.

##**Escribiendo archivos Excel**


In [19]:
# El metodo que nos permite guardar ---> se borrara cuando se cierre la sesion
#Aqui , de manera similar al csv , usamos index=False
intervalo.to_excel('co2_percapita.xlsx', index=False)

In [20]:
pd.read_excel('co2_percapita.xlsx')

,País,ISO 3166-1 alpha-3,Año,Total
0,Afganistán,AFG,1750,0.000000e+00
1,Afganistán,AFG,1751,0.000000e+00
2,Afganistán,AFG,1752,0.000000e+00
3,Afganistán,AFG,1753,0.000000e+00
4,Afganistán,AFG,1754,0.000000e+00
...,...,...,...,...
63099,Global,WLD,2017,3.609674e+10
63100,Global,WLD,2018,3.682651e+10
63101,Global,WLD,2019,3.708256e+10
63102,Global,WLD,2020,3.526409e+10


### 📤 Exportación a Excel (`to_excel`)

**Concepto de Negocio:**
La entrega final de valor en muchos proyectos de datos corporativos es un reporte en Excel. Al igual que en los archivos CSV, la consistencia de la API de Pandas nos permite usar una sintaxis casi idéntica para exportar.

**Buenas Prácticas (Clean Code):**
1. **`index=False`:** Regla de oro para evitar que el índice interno del sistema se imprima como una columna de basura en el reporte gerencial.
2. **`sheet_name`:** Aunque no es obligatorio, es una excelente práctica nombrar la hoja interna del Excel resultante (ej. 'Reporte_Mensual') en lugar del nombre genérico 'Sheet1' que pone Pandas por defecto. Esto da un acabado profesional al documento entregable.

In [21]:
import pandas as pd

# 1. Definición de Rutas (Tu Google Drive)
RUTA_EXPORTACION_EXCEL = '/content/drive/MyDrive/Pandas/reporte_co2_gerencial.xlsx'

# 2. Exportación de Datos
# Asumimos que usamos df_emisiones_resumen (el que filtramos por columnas A:D)
df_emisiones_resumen.to_excel(
    RUTA_EXPORTACION_EXCEL,
    index=False,
    sheet_name='Resumen_Directorio' # Nombrada profesionalmente
)

print(f"✅ Reporte Excel generado exitosamente en: {RUTA_EXPORTACION_EXCEL}")

# 3. Auditoría de Verificación (Leer lo que acabamos de escribir)
df_verificacion_excel = pd.read_excel(RUTA_EXPORTACION_EXCEL)
print("\n--- Vista previa del reporte final exportado ---")
display(df_verificacion_excel.head())

✅ Reporte Excel generado exitosamente en: /content/drive/MyDrive/Pandas/reporte_co2_gerencial.xlsx

--- Vista previa del reporte final exportado ---


,País,ISO 3166-1 alpha-3,Año,Total
0,Afganistán,AFG,1750,0
1,Afganistán,AFG,1751,0
2,Afganistán,AFG,1752,0
3,Afganistán,AFG,1753,0
4,Afganistán,AFG,1754,0


### ⚠️ Peligro de Sobrescritura: Agregando múltiples hojas a un Excel

**Concepto de Negocio:**
Si utilizamos `df.to_excel('archivo.xlsx')` de forma repetida sobre el mismo archivo, Pandas sobrescribirá el documento entero perdiendo las hojas anteriores.

**Estrategia de Solución (ExcelWriter):**
Para crear reportes gerenciales con múltiples pestañas (ej. "Ventas 2024" y "Ventas 2025" en el mismo libro), debemos utilizar el motor `pd.ExcelWriter` en modo "anexar" (`mode='a'`). Esto abre el archivo de forma segura, agrega la nueva hoja y lo vuelve a cerrar sin destruir la información previa.

In [24]:
# 1. Usamos el bloque 'with' (Gestor de contexto) para abrir y cerrar el archivo de forma segura.
# mode='a' significa "Append" (Anexar). engine='openpyxl' es el motor que permite esta acción.

with pd.ExcelWriter(RUTA_EXPORTACION_EXCEL, mode='a', engine='openpyxl') as gestor_excel:

    # 2. En lugar de pasarle la RUTA al to_excel, le pasamos el 'gestor_excel'
    datos_primeras_lineas.to_excel(
        gestor_excel,
        index=False,
        sheet_name='Primeras_lineas'
    )

print("✅ Nueva hoja agregada exitosamente sin borrar las anteriores.")

✅ Nueva hoja agregada exitosamente sin borrar las anteriores.


### ⏱️ Auditoría de Rendimiento Avanzada: El costo del modo Anexar (`mode='a'`)

**Observación Empírica:**
Al agregar una hoja con apenas 5 registros a un libro de Excel existente, el proceso demoró entre 10 y 12 segundos, casi lo mismo que exportar un DataFrame masivo.

**Concepto Arquitectónico (Overhead de I/O):**
Los archivos `.xlsx` son estructuralmente carpetas comprimidas (`.zip`) de código XML. Cuando usamos `ExcelWriter` en modo `append`, el motor (`openpyxl`) no puede simplemente "pegar" datos al final del archivo físico.
Debe ejecutar un proceso costoso:
1. Cargar y descomprimir el libro original completo en la RAM.
2. Mapear todo el XML existente.
3. Inyectar la nueva hoja (operación casi instantánea).
4. Volver a compilar, comprimir y guardar el archivo masivo resultante.

**Conclusión de Clean Code:**
El tiempo consumido es el "overhead" de manipular el archivo pesado preexistente, no el volumen de los datos nuevos. En sistemas de alta transaccionalidad (ej. logs en tiempo real), Excel es ineficiente; se prefieren bases de datos relacionales o archivos de adición continua (CSV/JSON).

##**Leyendo datos de Google Sheets**

link: https://docs.google.com/spreadsheets/d/1MqAPByeMc27nwIsnPjb6HnPbR8JVrli6egEvOzJCOVc/edit?usp=sharing

De aqui tenemos que sacar el ID de nyuestra hoja compartida, es decir desde la barra despues de la d/ , hasta la siguiente slash

```
1MqAPByeMc27nwIsnPjb6HnPbR8JVrli6egEvOzJCOVc
```



In [33]:
sheet_id = '1MqAPByeMc27nwIsnPjb6HnPbR8JVrli6egEvOzJCOVc'
#Tambien guardaremos el URL original completo
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/edit?usp=sharing'
# pero debemos ademas cambiar la parte de edit sharing por la api de google
# sheets (/gviz)para indicar que conectamos la planilla de google colab con el
# google sheets
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet'



In [34]:
datos_co2_sheets = pd.read_csv(url)

In [28]:
display(datos_co2_sheets)

,País,ISO 3166-1 alpha-3,Año,Total,Carbón,Aceite,Gas,Cemento,Quema,Otros,Per Capita
0,Afganistán,AFG,1750,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afganistán,AFG,1751,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afganistán,AFG,1752,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afganistán,AFG,1753,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afganistán,AFG,1754,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
63099,Global,WLD,2017,36.096.739.276,14.506.973.805,12.242.627.935,7.144.928.128,1.507.923.185,391.992.176,302.294.047,4.749.682
63100,Global,WLD,2018,36.826.506.600,14.746.830.688,12.266.016.285,7.529.846.784,1.569.218.392,412.115.746,302.478.706,4.792.753
63101,Global,WLD,2019,37.082.558.969,14.725.978.025,12.345.653.374,7.647.528.220,1.617.506.786,439.253.991,306.638.573,4.775.633
63102,Global,WLD,2020,35.264.085.734,14.174.564.010,11.191.808.551,7.556.290.283,1.637.537.532,407.583.673,296.301.685,4.497.423


In [32]:
#aqui agregaremos a nuestro link , el parametro de sheetname
sheet_id = '1MqAPByeMc27nwIsnPjb6HnPbR8JVrli6egEvOzJCOVc'
sheet_name = 'emisiones_percapita'
#entonces ahora tenemos parametrizada la extraccion de cada hoja
url_percapita = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
#procedemos a leer
percapita_sheets = pd.read_csv(url_percapita)
#mostramos
display(percapita_sheets)

,País,ISO 3166-1 alpha-3,Año,Total,Carbón,Aceite,Gas,Cemento,Quema,Otros
0,Afganistán,AFG,1750,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afganistán,AFG,1751,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afganistán,AFG,1752,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afganistán,AFG,1753,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afganistán,AFG,1754,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
63099,Global,WLD,2017,4.749.682,1.908.857,1.610.910,940.144,198.416,51.579,39.776
63100,Global,WLD,2018,4.792.753,1.919.213,1.596.350,979.965,204.225,53.634,39.366
63101,Global,WLD,2019,4.775.633,1.896.468,1.589.920,984.878,208.309,56.569,39.490
63102,Global,WLD,2020,4.497.423,1.807.760,1.427.353,963.695,208.844,51.981,37.789


### ☁️ Ingesta desde la Nube: Google Sheets API

**Concepto de Negocio:**
Para automatizar la lectura de un Google Sheet en Pandas sin usar complejas librerías de autenticación (OAuth), podemos utilizar el punto de conexión público de *Google Visualization API*.

**El Truco de Arquitectura:**
1. El documento debe estar configurado como "Cualquier usuario que tenga el vínculo puede leer".
2. No usamos `read_excel`. Construimos una URL especial que le pide a Google que exporte la pestaña específica a un formato texto en tiempo real (`out:csv`) y la leemos directamente con `pd.read_csv`.

**Buenas Prácticas (Clean Code):**
En lugar de copiar y pegar URLs gigantes por cada pestaña (lo cual viola el principio DRY: *Don't Repeat Yourself*), encapsularemos la lógica de la URL en una función en Python.

In [29]:
import pandas as pd

# 1. El "DNI" de tu documento (Lo sacas de tu propia URL de Google Sheets)
# Nota: Reemplaza esto con el ID real de TU archivo si creaste uno.
ID_DOCUMENTO = '1MqAPByeMc27nwIsnPjb6HnPbR8JVrli6egEvOzJCOVc'

# 2. Función Generadora de URLs (Clean Code)
def obtener_url_google_sheet(id_doc, nombre_hoja):
    """
    Construye la URL de la API de Google para descargar una pestaña como CSV.
    """
    url_base = f'https://docs.google.com/spreadsheets/d/{id_doc}/gviz/tq'
    parametros = f'?tqx=out:csv&sheet={nombre_hoja}'
    return url_base + parametros

# -------------------------------------------------------------------
# 3. Ingesta Elegante (Una sola línea por pestaña)
# -------------------------------------------------------------------

# Descargar Hoja 1
url_hoja1 = obtener_url_google_sheet(ID_DOCUMENTO, 'emisiones_CO2')
df_emisiones_nube = pd.read_csv(url_hoja1)

# Descargar Hoja 2
url_hoja2 = obtener_url_google_sheet(ID_DOCUMENTO, 'emisiones_percapita')
df_percapita_nube = pd.read_csv(url_hoja2)

# 4. Auditoría
print("✅ Datos descargados exitosamente desde la nube.")
print("\n--- Vista previa de Emisiones Per Cápita ---")
display(df_percapita_nube.head(3))

✅ Datos descargados exitosamente desde la nube.

--- Vista previa de Emisiones Per Cápita ---


,País,ISO 3166-1 alpha-3,Año,Total,Carbón,Aceite,Gas,Cemento,Quema,Otros
0,Afganistán,AFG,1750,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afganistán,AFG,1751,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afganistán,AFG,1752,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Apuntes Adicionales

## Ejercicio Aula -2



### 🏋️‍♂️ Desafío Práctico: Ingesta desde URL y Exportación a CSV

**Contexto del Negocio:**
Para expandir nuestro análisis de sostenibilidad global, necesitamos procesar un nuevo conjunto de datos. Este dataset documenta las emisiones de CO2 per cápita de todos los países del mundo durante el periodo 1990 - 2019 (fuente original: Kaggle).

**Objetivo:**
1. **Extracción (Extract):** Leer los datos directamente desde el enlace web proporcionado.
2. **Carga (Load):** Guardar el DataFrame obtenido en nuestro sistema (Google Drive) en formato de texto plano (`.csv`).

**Recursos:**
* **Enlace de los Datos:** [Emisiones de Carbono Mundo](https://cdn3.gnarususercontent.com.br/1980-pandas-e-s/emisiones_carbono_mundo.xlsx)

### 🛡️ Superando Bloqueos de Seguridad (Error 403 Forbidden)

**El Problema:**
Al intentar leer un archivo directamente desde una URL externa, es común recibir un `HTTP Error 403`. Esto ocurre porque los servidores modernos tienen firewalls que bloquean solicitudes automatizadas (bots) que provienen de lenguajes de programación en lugar de navegadores web estándar.

**La Solución (Modificación del User-Agent):**
Utilizamos el parámetro `storage_options` de la función `read_excel` para inyectar una cabecera HTTP falsa (`User-Agent: Mozilla/5.0`). Esto engaña al servidor, haciéndole creer que la petición de descarga proviene de un navegador web humano normal, permitiendo que la descarga fluya hacia nuestro DataFrame.

In [36]:
import pandas as pd

# 1. Definición de Rutas y Constantes
RUTA_ARCHIVO = 'https://cdn3.gnarususercontent.com.br/1980-pandas-e-s/emisiones_carbono_mundo.xlsx'
RUTA_EXPORTACION_CSV = '/content/drive/MyDrive/Pandas/emisiones_co2_mundo.csv'

# 2. Configuración de Seguridad (El "Disfraz" de Navegador)
# Diccionario con las cabeceras HTTP que inyectaremos
opciones_conexion = {'User-Agent': 'Mozilla/5.0'}

# 3. Ingesta de Datos (Superando el firewall)
df_co2_mundo = pd.read_excel(
    RUTA_ARCHIVO,
    storage_options=opciones_conexion  # Pasamos nuestro diccionario aquí
)

# 4. Transformación y Carga (Guardado local)
df_co2_mundo.to_csv(RUTA_EXPORTACION_CSV, index=False)

# 5. Inspección de Calidad
print(f"✅ Éxito: Firewall superado. Archivo guardado en {RUTA_EXPORTACION_CSV}")
print("\n--- Auditoría QA: Emisiones de CO2 en el Mundo ---")
display(df_co2_mundo.head())

✅ Éxito: Firewall superado. Archivo guardado en /content/drive/MyDrive/Pandas/emisiones_co2_mundo.csv

--- Auditoría QA: Emisiones de CO2 en el Mundo ---


,Country Name,country_code,Region,Indicator Name,1990,1991,1992,1993,1994,1995,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2019.1
0,Aruba,ABW,Latin America & Caribbean,CO2 emissions (metric tons per capita),NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,South Asia,CO2 emissions (metric tons per capita),1.917451e+08,167681579.0,95957742.0,8.472111e+06,7.554583e+07,6.846796e+07,...,2.965062e+07,2.592953e+08,1.856237e+07,1.462356e+08,1.728967e+08,1.497893e+08,1.316946e+08,1.632953e+06,1.598244e+08,1.598244e+08
2,Angola,AGO,Sub-Saharan Africa,CO2 emissions (metric tons per capita),5.536620e+08,544538649.0,543557223.0,7.089842e+07,8.368044e+08,9.121415e+08,...,9.855223e+08,9.506959e+08,1.036294e+09,1.099779e+09,1.135044e+08,1.031811e+09,8.133007e+07,7.776749e+08,7.921371e+08,7.921371e+08
3,Albania,ALB,Europe & Central Asia,CO2 emissions (metric tons per capita),1.819542e+09,124281022.0,683699826.0,6.383070e+08,6.453552e+08,6.054363e+08,...,1.669423e+08,1.503240e+08,1.533630e+09,1.668337e+09,1.603775e+08,1.557664e+09,1.788786e+09,1.782739e+09,1.692248e+08,1.692248e+08
4,Andorra,AND,Europe & Central Asia,CO2 emissions (metric tons per capita),7.521832e+09,723537924.0,69630787.0,6.724178e+09,6.541579e+09,6.733479e+09,...,5.850886e+09,5.944654e+09,5.942800e+09,5.807128e+09,6.026182e+09,6.080600e+09,6.104134e+09,6.362975e+09,6.481217e+09,6.481217e+09
